In [14]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd
import numpy as np
import transformations  as tr
import random as rand
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

ImportError: Traceback (most recent call last):
  File "C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\__init__.py", line 54, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\Lucas\Ap

In [2]:
def generatePairs(frames):
    framePairs = [] #vector to hold image pairs

    for x in range(len(frames)): #x represents the first frame in the pair
        for y in range(len(frames)-3-x): #y represents second frame in the pair
            pair = [(x+1),(x+y+4),np.stack((frames[x], frames[x+y+3]), axis=-1)] #generate pair
            framePairs.append(pair)
    return framePairs

In [3]:
def translatePair(pair,steps,patient):

    frame1 = pair[0]-1 #get first frame position from pair object
    frame2 = pair[1]-1 #get second frame position from pair object
    JAdd = pair[2].shape[0] #Get size of original frame to keep translation size consistent
    IAdd = pair[2].shape[1] #Get size of original frame to keep translation size consistent

    baseCut = 0.1 #use same base cut as is used in original frame generation
    step = baseCut/steps #set step size to be as large as possible for given number of steps
    pairs = [] #vector to hold newly generated pairs

    #adjust how the frame is cropped from the original data to simulate translation
    for x in range(-steps,steps+1): 
        for y in range(-steps,steps+1):
            lowerI = int((cutLeft+x*step)*256)
            upperI = lowerI+IAdd
            lowerJ = int((cutTop+y*step)*2592)
            upperJ = lowerJ+JAdd

            first = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame1]
            second = rawData[patient][1][lowerJ:upperJ,lowerI:upperI,frame2]

            pairs.append(np.stack((first,second), axis=-1)) #append translated pair to pairs vector
    return pairs

In [4]:
def normalize(min,max,array):
    test[test < min] = min
    test[test > max] = max
    array = tr.unit_vector(array)
    return array

In [5]:
#Directory containing the RF scan data
dataFolder = 'C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/'

#Vector to hold all training data sets
rawData = []

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder+'P39-W2-S4')
#Contains just the RF data pulled from the .mat file
P39W2S4 = rawMatFile['rf1']
rawData.append(['P39W2S4',P39W2S4])

#repeat above steps for other RF files
rawMatFile = loadmat(dataFolder+'P39-W4-S6')
P39W4S6 = rawMatFile['rf1']
rawData.append(['P39W4S6',P39W4S6])
rawMatFile = loadmat(dataFolder+'P82-W0-S2')
P82W0S2 = rawMatFile['rf1']
rawData.append(['P82W0S2',P82W0S2])
rawMatFile = loadmat(dataFolder+'P87-W0-S3')
P87W0S3 = rawMatFile['rf1']
rawData.append(['P87W0S3',P87W0S3])
rawMatFile = loadmat(dataFolder+'P87-W2-S4')
P87W2S4 = rawMatFile['rf1']
rawData.append(['P87W2S4',P87W2S4])
rawMatFile = loadmat(dataFolder+'P90-W0-S4')
P90W0S4 = rawMatFile['rf1']
rawData.append(['P90W0S4',P90W0S4])
rawMatFile = loadmat(dataFolder+'P94-W1-S3')
P94W1S3 = rawMatFile['rf1']
rawData.append(['P94W1S3',P94W1S3])

In [6]:
#leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

allFrames = [] #vector to hold individual cropped frames

for y in range(len(rawData)):
    frames = []
    for x in range(rawData[y][1].shape[2]): #range is how many frames there are in the set
        frames.append(rawData[y][1][lowerJ:upperJ,lowerI:upperI,x]) #Take inner subset of frame to allow for translation
    allFrames.append([rawData[y][0],frames])

pairs = []
for x in range(len(allFrames)):
    pairs.append([allFrames[x][0],generatePairs(allFrames[x][1])]) #add generated pairs to list
print(pairs[6][1][0][2].shape)

#PAIRS VARIABLE GUIDE
#First value = Holds which patient set is being referred to
#Second value = Holds either patient set name [0] or generated pairs for set
#Third value = Holds all pairs for patient set
#Fourth value = holds specific pairs for patient set
#pairs[0][1][0][2].shape <- references the shape of the third pair of frames for a certain patients data

(1805, 205, 2)


In [7]:
#Create Vector of stacked pairs with frame number and patient ID for easy labeling
data = []
for x in range(len(pairs)):
    for y in range(len(pairs[x][1])):
        data.append([pairs[x][0],pairs[x][1][y][0],pairs[x][1][y][1],pairs[x][1][y][2]])

print(len(data))
#for x in range(200):
#    print(data[x][0],data[x][1],data[x][2],data[x][3].shape)

1134


In [8]:
#Read in label data
DFP39W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P39-W2-S4', engine='openpyxl')
DFP39W4S6 = pd.read_excel('Labels.xlsx', sheet_name='P39-W4-S6', engine='openpyxl')
DFP82W0S2 = pd.read_excel('Labels.xlsx', sheet_name='P82-W0-S2', engine='openpyxl')
DFP87W0S3 = pd.read_excel('Labels.xlsx', sheet_name='P87-W0-S3', engine='openpyxl')
DFP87W2S4 = pd.read_excel('Labels.xlsx', sheet_name='P87-W2-S4', engine='openpyxl')
DFP90W0S4 = pd.read_excel('Labels.xlsx', sheet_name='P90-W0-S4', engine='openpyxl')
DFP94W1S3 = pd.read_excel('Labels.xlsx', sheet_name='P94-W1-S3', engine='openpyxl')

In [9]:
#create labels for training data
labels = DFP39W2S4['Label axial'].tolist() + DFP39W4S6['Label axial'].tolist() + DFP82W0S2['Label axial'].tolist() + DFP87W0S3['Label axial'].tolist() + DFP87W2S4['Label axial'].tolist() + DFP90W0S4['Label axial'].tolist() + DFP94W1S3['Label axial'].tolist()
print(len(labels))

1134


In [10]:
labeledData = []
for x in range(len(labels)):
    labeledData.append([data[x][3],labels[x]])

In [11]:
goodData = []
badData = []
for x in range(len(labeledData)):
    if labeledData[x][1] > 0:
        goodData.append(labeledData[x])
    else:
        badData.append(labeledData[x])
print(len(goodData))
print(len(badData))
badData = rand.sample(badData,len(goodData))
print(len(goodData))
print(len(badData))
balancedData = goodData+badData
print(type(balancedData))

240
894
240
240
<class 'list'>


In [164]:
# Split data in the ratio 80-20 for training and validation.
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in balancedData], [x[1] for x in balancedData], test_size=0.20, random_state=42)

In [162]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

384
